In [10]:
import os
def create_directory_recursively(path):
    try:
        # exist_ok=True 옵션은 해당 경로의 폴더가 이미 존재해도 에러를 발생시키지 않음
        os.makedirs(path, exist_ok=True)
        print(f"'{path}' 디렉토리가 생성되었거나 이미 존재합니다.")
    except PermissionError:
        # 권한 오류 처리
        print(f"권한 오류: '{path}' 디렉토리를 생성할 수 없습니다. 접근 권한을 확인하세요.")
    except Exception as e:
        # 기타 오류 처리
        print(f"오류 발생: {e}")
def find_files(root_directory, partial_name):
    """
    주어진 루트 디렉토리에서 부분 이름을 포함하는 모든 파일의 경로를 찾습니다.

    :param root_directory: 검색을 시작할 루트 디렉토리
    :param partial_name: 찾고자 하는 파일 이름의 일부
    :return: 찾은 파일의 전체 경로 목록
    """
    matching_files = []
    for root, dirs, files in os.walk(root_directory):
        for file in files:
            if partial_name in file:
                matching_files.append(os.path.join(root, file))
    return matching_files
def is_point_inside_bbox(bbox, point):
    """
    주어진 bbox 내에 특정 좌표가 포함되어 있는지 확인합니다.

    :param bbox: Bounding box의 좌표 (x_min, y_min, x_max, y_max)
    :param point: 확인할 좌표 (x, y)
    :return: 좌표가 bbox 내에 있으면 True, 아니면 False
    """
    x_min, y_min, x_max, y_max = bbox
    x, y = point

    return x_min <= x <= x_max and y_min <= y <= y_max


In [33]:
import os
def count_mp4_files(path,end):
  """
  주어진 경로와 하위 폴더들을 재귀적으로 탐색하여 mp4 파일 개수를 센다.

  Args:
    path: 탐색을 시작할 경로

  Returns:
    mp4 파일 개수
  """
  mp4_files = []
  mp4_count = 0
  def _count_mp4_files_in_dir(dir_path):
    nonlocal mp4_count
    nonlocal mp4_files
    for file in os.listdir(dir_path):
      full_path = os.path.join(dir_path, file)
      if os.path.isfile(full_path):
        if file.endswith(f".{end}") and not "매장이동" in full_path:
          mp4_count += 1
          mp4_files.append(full_path)
      elif os.path.isdir(full_path):
        _count_mp4_files_in_dir(full_path)

  _count_mp4_files_in_dir(path)

  return mp4_count, mp4_files

# 예시 코드
path = "/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/01-1.정식개방데이터"
xml_count, xml_files = count_mp4_files(path,"xml")
print(xml_count)

4670


In [3]:
import json
import xmltodict



In [22]:
import pickle
data=[]
with open("output.pkl", 'wb') as file2:
    pickle.dump(data, file2)
with open("output.pkl", 'rb') as file:
    data = pickle.load(file)
print(data)

[]


In [35]:
import copy
import pickle
import numpy as np  
root_directory = "/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/PreProcessToPkl" # 검색을 시작할 루트 디렉토리 경로로 변경하세요.
error_list=[]
action_list= [
    "select_start","select_end",
    "test_start", "test_end",
    "buying_start", "buying_end",
    "return_start", "return_end",
    "compare_start", "compare_end"
    ]
for idx, xml_file in enumerate(xml_files):
    print("-------------------",idx)
    with open(xml_file, 'r', encoding='utf-8') as file:
        xml_string = file.read()
        # XML 문자열을 파이썬 딕셔너리로 변환
    dict_data = xmltodict.parse(xml_string)
    actionbbox_list=[]
    for i in dict_data['annotations']["track"]:
        if i["@label"] in action_list:
            if "start" in i["@label"]:
                if(type(i['box'])==list):
                    bbox = (i["@label"] , i['box'][0]["@frame"] ,(float(i['box'][0]["@xtl"]),float(i['box'][0]["@ytl"]),float(i['box'][0]["@xbr"]),float(i['box'][0]["@ybr"])))
                    actionbbox_list.append(bbox)
                else:
                    bbox = (i["@label"] , i['box']["@frame"] ,(float(i['box']["@xtl"]),float(i['box']["@ytl"]),float(i['box']["@xbr"]),float(i['box']["@ybr"])))
                    actionbbox_list.append(bbox)
    pkl_path = copy.deepcopy(xml_file)
    pklsplit=pkl_path.split("/")[-1][:-4]
    found_files = find_files(root_directory, pklsplit)
    if(len(found_files)!=len(actionbbox_list)):
        print(len(found_files))
        print(found_files)
        error_list.append(found_files)
        print(len(actionbbox_list))
        print(actionbbox_list)
        continue
    for idx , found_file in enumerate(found_files):
        with open(found_file, 'rb') as file:
            data = pickle.load(file)
            newfilepath=found_file.replace("PreProcessToPkl","PreProcessToPkl2")
            if(os.path.exists(newfilepath)):
                continue
            if(data["keypoint"].shape[0]==1):
                continue
            for idx2,j in enumerate(data["keypoint"]):
                frame,label,bbox =actionbbox_list[idx]
                point = tuple(j[0][0])
                if is_point_inside_bbox(bbox, point):
                    new_video_dir = os.path.dirname(newfilepath)
                    create_directory_recursively(new_video_dir)
                    data["keypoint"] = np.expand_dims(j,axis=0)
                    with open(newfilepath, 'wb') as file2:
                        pickle.dump(data, file2)
                else:
                    continue


------------------- 0
------------------- 1
------------------- 2
------------------- 3
------------------- 4
------------------- 5
------------------- 6
------------------- 7
------------------- 8
------------------- 9
------------------- 10
------------------- 11
------------------- 12
------------------- 13
------------------- 14
------------------- 15
------------------- 16
------------------- 17
------------------- 18
------------------- 19
------------------- 20
------------------- 21
------------------- 22
------------------- 23
------------------- 24
------------------- 25
------------------- 26
------------------- 27
------------------- 28
------------------- 29
------------------- 30
------------------- 31
------------------- 32
------------------- 33
------------------- 34
------------------- 35
------------------- 36
------------------- 37
------------------- 38
------------------- 39
------------------- 40
------------------- 41
------------------- 42
------------------- 4

In [13]:
with open('/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/PreProcessToPkl/Validation/01.원천데이터/VS_02.구매행동_06.비교/C_2_6_55_BU_DYB_10-23_15-27-36_CA_RGB_DF1_F3_F3_116_243.pkl', 'rb') as file:
    data = pickle.load(file)
data["keypoint"].shape

(7, 128, 17, 2)

In [76]:
print(data["keypoint"].shape)
print(data["keypoint"][0].shape)



(1, 24, 17, 2)
(24, 17, 2)
[758.3477  411.09702]


In [71]:
tuple(data["keypoint"][0][0][0])
frame,label,bbox =actionbbox_list[0]
bbox

(486.67, 223.33, 975.0, 1031.7)

In [72]:

# 사용 예시
point = tuple(data["keypoint"][0][0][0])  # 확인할 좌표

if is_point_inside_bbox(bbox, point):
    print("주어진 좌표는 bbox 내에 있습니다.")
else:
    print("주어진 좌표는 bbox 밖에 있습니다.")


주어진 좌표는 bbox 내에 있습니다.


In [23]:
erro_xml_list=[]
erro_video_list=[]
with open("output_xml_error.pkl", 'rb') as file2:
    erro_xml_list =pickle.load(file2)
with open("output_video_error.pkl", 'rb') as file2:
    erro_video_list =pickle.load(file2)


In [25]:
len(erro_video_list)

5

In [26]:
len(erro_xml_list)

5

In [27]:
erro_xml_list

['/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/01-1.정식개방데이터/Validation/02.라벨링데이터/VL_02.구매행동_05.반품/C_2_5_113_BU_DYB_10-20_16-00-20_CA_RGB_DF1_M4_M4.xml',
 '/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/01-1.정식개방데이터/Training/02.라벨링데이터/TL_02.구매행동_05.반품/C_2_5_40_BU_DYB_10-10_13-40-04_CB_RGB_DF1_F2_F2.xml',
 '/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/01-1.정식개방데이터/Training/02.라벨링데이터/TL_02.구매행동_05.반품/C_2_5_14_BU_DYB_10-10_11-39-08_CC_RGB_DF1_F1_F1.xml',
 '/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/01-1.정식개방데이터/Training/02.라벨링데이터/TL_02.구매행동_05.반품/C_2_5_1_BU_DYB_10-10_11-25-04_CC_RGB_DF1_M1_M1.xml',
 '/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/01-1.정식개방데이터/Training/02.라벨링데이터/TL_02.구매행동_05.반품/C_2_5_33_BU_DYB_10-10_12-49-14_CA_RGB_DF1_M2_M2.xml']

In [38]:
import copy
import pickle
import numpy as np  
root_directory = "/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/PreProcessToPkl" # 검색을 시작할 루트 디렉토리 경로로 변경하세요.
error_list=[]
action_list= [
    "select_start","select_end",
    "test_start", "test_end",
    "buying_start", "buying_end",
    "return_start", "return_end",
    "compare_start", "compare_end"
    ]
for idx, xml_file in enumerate(xml_files):
    print("-------------------",idx)
    print(xml_file)
    with open(xml_file, 'r', encoding='utf-8') as file:
        xml_string = file.read()
        # XML 문자열을 파이썬 딕셔너리로 변환
    dict_data = xmltodict.parse(xml_string)
    actionbbox_list=[]
    for i in dict_data['annotations']["track"]:
        if i["@label"] in action_list:
            if "start" in i["@label"]:
                if(type(i['box'])==list):
                    bbox = (i["@label"] , i['box'][0]["@frame"] ,(float(i['box'][0]["@xtl"]),float(i['box'][0]["@ytl"]),float(i['box'][0]["@xbr"]),float(i['box'][0]["@ybr"])))
                    actionbbox_list.append(bbox)
                else:
                    bbox = (i["@label"] , i['box']["@frame"] ,(float(i['box']["@xtl"]),float(i['box']["@ytl"]),float(i['box']["@xbr"]),float(i['box']["@ybr"])))
                    actionbbox_list.append(bbox)
    pkl_path = copy.deepcopy(xml_file)
    pklsplit=pkl_path.split("/")[-1][:-4]
    found_files = find_files(root_directory, pklsplit)
    if(len(found_files)!=len(actionbbox_list)):
        print(len(found_files))
        print(found_files)
        error_list.append(found_files)
        print(len(actionbbox_list))
        print(actionbbox_list)
        continue
    for idx , found_file in enumerate(found_files):
        with open(found_file, 'rb') as file:
            data = pickle.load(file)
            newfilepath=found_file.replace("PreProcessToPkl","PreProcessToPkl3")
            if(os.path.exists(newfilepath)):
                continue
            if(data["keypoint"].shape[0]==1):
                continue
            keypoint = np.zeros((30, 1, 17, 2),dtype=np.float16)
            data["keypoint"] = data["keypoint"].transpose((1, 0, 2, 3))
            for idx2,j in enumerate(data["keypoint"]):
                frame,label,bbox =actionbbox_list[idx]
                point = tuple(j[0][0])
                if is_point_inside_bbox(bbox, point):
                    new_video_dir = os.path.dirname(newfilepath)
                    create_directory_recursively(new_video_dir)
                    data["keypoint"] = np.expand_dims(j,axis=0)
                    # with open(newfilepath, 'wb') as file2:
                        # pickle.dump(data, file2)
                        # pass
                else:
                    continue
        break
    break


------------------- 0
/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/01-1.정식개방데이터/Validation/02.라벨링데이터/VL_02.구매행동_06.비교/C_2_6_55_BU_DYB_10-23_15-27-36_CB_RGB_DF1_F3_F3.xml


In [39]:
xml_file="/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/01-1.정식개방데이터/Validation/02.라벨링데이터/VL_02.구매행동_06.비교/C_2_6_55_BU_DYB_10-23_15-27-36_CB_RGB_DF1_F3_F3.xml"
with open(xml_file, 'r', encoding='utf-8') as file:
    xml_string = file.read()
    # XML 문자열을 파이썬 딕셔너리로 변환
dict_data = xmltodict.parse(xml_string)
actionbbox_list=[]
for i in dict_data['annotations']["track"]:
    if i["@label"] in action_list:
        if "start" in i["@label"]:
            if(type(i['box'])==list):
                bbox = (i["@label"] , i['box'][0]["@frame"] ,(float(i['box'][0]["@xtl"]),float(i['box'][0]["@ytl"]),float(i['box'][0]["@xbr"]),float(i['box'][0]["@ybr"])))
                actionbbox_list.append(bbox)
            else:
                bbox = (i["@label"] , i['box']["@frame"] ,(float(i['box']["@xtl"]),float(i['box']["@ytl"]),float(i['box']["@xbr"]),float(i['box']["@ybr"])))
                actionbbox_list.append(bbox)
print(actionbbox_list)


[('compare_start', '116', (893.3, 260.0, 1215.0, 728.3)), ('compare_start', '447', (879.54, 276.41, 1179.31, 733.59))]


In [ ]:
Right foot  ,  Spine chest

In [46]:
xml_file="/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/01-1.정식개방데이터/Validation/02.라벨링데이터/VL_02.구매행동_06.비교/C_2_6_55_BU_DYB_10-23_15-27-36_CB_RGB_DF1_F3_F3.xml"
with open(xml_file, 'r', encoding='utf-8') as file:
    xml_string = file.read()
    # XML 문자열을 파이썬 딕셔너리로 변환
dict_data = xmltodict.parse(xml_string)
all_list=[]
batch_list = []
print(dict_data['annotations']["track"][0]["@label"])
for i in dict_data['annotations']["track"]:
    batch_list.append(i)
    if(i["@label"]=="Spine chest"):
        all_list.append(batch_list)
print(all_list)